In [ ]:
# Import packages

import pandas as pd
import urllib.request, json

In [ ]:
# Etherscan API call
# Powered by Etherscan.io APIs

def es_data(es_address, es_key):
    """Pulls Ether balance from a specified address (es_address) using the Etherscan API. Requires a user-specific API key (es_key).
    
    Args:
        es_address: Ether address, the public key for an Ether account.
        es_key: Personal EtherScan API key, required for calls to their API.
    
    Returns:
        JSON with status of API call and balance of ETH tokens associated with Ether address.
    
    """
    
    es_url = "https://api.etherscan.io/api?module=account&action=balance&address=%s&tag=latest&apikey=%s" % (es_address, es_key)
    with urllib.request.urlopen(es_url) as url:
        output = json.loads(url.read().decode())
    return output

In [ ]:
# CoinMarketCap API call
# Powered by CoinMarketCap APIs

def cmc_data(cmc_key):
    """Pulls Ether price information.
    
    Args:
        cmc_key: Personal CoinMarketCap API key, required for calls to their API.
        
    Returns:
        JSON with latest aggregated trading information for ETH token.
    
    """
    
    cmc_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?symbol=ETH&convert=CAD&CMC_PRO_API_KEY=%s" % (cmc_key)
    with urllib.request.urlopen(cmc_url) as url:
        output = json.loads(url.read().decode())
    return output

In [ ]:
# Get quantity of ETH held by specified address

def get_quantity():
    """Returns the ETH balance of address from Etherscan API call.
    
    Returns:
        Number of ETH tokens in specified Ether address.
        
    Raises:
        ValueError: if EtherScan API call was unsuccessful.
    
    """
    
    try:
        if int(es_output["status"]) != 1:
            raise ValueError("Error: API call was unsuccessful. Message: {}".format(es_output["message"]))
    except ValueError as ve:
        quantity = "Error"
        print(ve)
    else:
        quantity = float(es_output["result"]) / 1.0e18
    return quantity

In [ ]:
# Get latest price and update time for ETH

def get_price_info():
    """Returns ETH price infomration from CoinMarketCap API call.
    
    Returns:
       Latest ETH price (in CAD) and the date and time it was last updated.
       
    Raises:
        ValueError: if CoinMarketCap API call was unsuccessful.
    
    """
    
    try:
        if int(cmc_output["status"]["error_code"]) != 0:
            raise ValueError("Error: API call was unsuccessful. Message: {}".format(cmc_output["status"]["error_message"]))
    except ValueError as ve:
        price = "Error"
        updated_time = "Error"
        print(ve)
    else:
        price = float(cmc_output["data"]["ETH"]["quote"]["CAD"]["price"])
        update_time = cmc_output["data"]["ETH"]["quote"]["CAD"]["last_updated"] 
    return (price, update_time)

In [ ]:
# Calculate value of ETH holding

def holding_value(quantity, price):
    """Calculates the dollar value of ETH position based on quantity and price.
    
    Args:
        quantity: number of ETH tokens associated with Ether address.
        price: latest price per ETH token.
    
    Returns:
        Dollar value of ETH holding in CAD.
    
    """
    
    holding_value = round(quantity * price, 2)
    return holding_value

In [ ]:
# Create DataFrame to be exported to Excel spreadsheet

def create_df():
    """Creates a DataFrame with holding data.
    
    Returns:
        DataFrame with one record of ETH token quantity, price, holding value, and time of latest update.
    
    """
    
    df = pd.DataFrame({"Symbol":cmc_output["data"]["ETH"]["symbol"],
                       "Quantity":quantity,
                       "Price":price,
                       "Value":holding_value
                      }, index=[update_time])
    return df

In [ ]:
# Append DataFrame result to CSV

def df_to_csv(df):
    """Append DataFrame to CSV.
    
    Returns:
        Append DataFrame to existing CSV in new row.
    
    """
    
    file_name = "output.csv"
    with open(file_name, "a") as f:
        df.to_csv(f, header=False)

In [ ]:
# Run the code

es_address = input("EtherScan Address: ")
es_key = input("EtherScan API key: ")
cmc_key = input("CoinMarketCap API key: ")

es_output = es_data(es_address, es_key)
cmc_output = cmc_data(cmc_key)

quantity = get_quantity()
price, update_time = get_price_info()
holding_value = holding_value(quantity, price)

df = create_df()
df_to_csv(df)